# INST326 — Week 9 Exercises: Abstract Classes & Interfaces (Library Management)

**Focus (Week 9 only):** Abstract Base Classes (ABCs) with `abc.ABC` and `@abstractmethod`, abstract properties, virtual subclass registration, and light “interface-like” design via ABCs and (optional) `typing.Protocol` **without** advanced generics.

**Out of scope (Week 10+):** multiple inheritance/mixins, advanced design patterns, dependency injection, metaclasses beyond `ABCMeta`, decorators beyond basics, complex type-system features (ParamSpec, TypeVar variance), context managers beyond prior weeks.


### Starter Scaffold (Week-9-safe)

Below is a minimal domain model from prior weeks, slightly adapted for Week 9. We keep inheritance simple and introduce **abstract classes** to define common contracts.


In [ ]:
from __future__ import annotations
from dataclasses import dataclass
from datetime import datetime, timedelta
from typing import Optional, List, Dict, Protocol
from abc import ABC, abstractmethod


# --- Exceptions (kept simple) ---
class LibraryError(Exception): ...
class DuplicateBookError(LibraryError): ...
class OverdueLoanError(LibraryError): ...
class NonBorrowableError(LibraryError): ...


# --- Abstract base for library items ---
class LibraryItem(ABC):
    def __init__(self, isbn: str, title: str, copies: int = 1) -> None:
        self.isbn = isbn
        self.title = title
        self.copies = copies

    @abstractmethod
    def loan_period_days(self) -> int:
        """Each concrete item defines its loan period."""

    @abstractmethod
    def describe(self) -> str:
        """Human-readable description of the item."""

    @property
    @abstractmethod
    def is_digital(self) -> bool:
        """Whether the item is digital."""

    @property
    @abstractmethod
    def media_type(self) -> str:
        """Media type as string."""

    @classmethod
    @abstractmethod
    def kind(cls) -> str:
        """Briefly identify the kind of book."""

    @property
    @abstractmethod
    def borrowable(self) -> bool:
        """Whether or not the item is borrowable."""

    @abstractmethod
    def daily_late_fee(self) -> float:
        """Late fee per day for the given book."""

    @abstractmethod
    def validate_on_add(self) -> None:
        """Validate a book when adding."""
        
    # A partial template method that depends on an abstract method.
    def due_date_from_today(self) -> datetime:
        return datetime.now() + timedelta(days=self.loan_period_days())

    # default stock policy common to most items
    def can_checkout(self) -> bool:
        return self.copies > 0

    def receipt_line(self) -> str:
        return f"{self.describe()} — Due in {self.loan_period_days()} days"

    def late_fee(self, days_late: int) -> float:
        return self.daily_late_fee() * days_late

    def full_label(self) -> str:
        return f"[{self.media_type}] {self.title} — {self.isbn}"


# --- Concrete items (will be extended in exercises) ---
class PrintedBook(LibraryItem):
    def loan_period_days(self) -> int:
        return 21
    def describe(self) -> str:
        return f"PrintedBook<{self.isbn}>: {self.title} (copies={self.copies})"
    @property
    def is_digital(self) -> bool:
        return False
    @property
    def media_type(self) -> str:
        return "print"
    @classmethod
    def kind(cls) -> str:
        return "printed"
    @property
    def borrowable(self) -> bool:
        return True
    def daily_late_fee(self) -> float:
        return 0.25
    def validate_on_add(self) -> None:
        if self.copies < 0:
            raise ValueError("printed book copies must be non-negative")

class EBook(LibraryItem):
    def __init__(self, isbn: str, title: str, copies: int = 0, file_size_mb: float = 0.0) -> None:
        super().__init__(isbn, title, copies)
        self.file_size_mb = float(file_size_mb)
    def loan_period_days(self) -> int:
        return 14
    def describe(self) -> str:
        return f"EBook<{self.isbn}>: {self.title} ({self.file_size_mb:.1f} MB)"
    @property
    def is_digital(self) -> bool:
        return True
    # Different stock rule (licenses can be zero but not negative)
    def can_checkout(self) -> bool:
        return self.copies >= 0
    @property
    def media_type(self) -> str:
        return "ebook"
    @classmethod
    def kind(cls) -> str:
        return "ebook"
    @property
    def borrowable(self) -> bool:
        return self.copies >= 0
    def daily_late_fee(self) -> float:
        return 0.10
    def validate_on_add(self) -> None:
        if self.copies < 0:
            raise ValueError("ebook copies must be non-negative")
    def download_link(self) -> str:
        return f"https://downloads.example.com/{self.isbn}.epub"

class AudioBook(LibraryItem):
    def __init__(self, isbn: str, title: str, copies: int = 1, duration_min: int = 0) -> None:
        super().__init__(isbn, title, copies)
        self.duration_min = int(duration_min)
    def loan_period_days(self) -> int:
        return 14
    def describe(self) -> str:
        return f"AudioBook<{self.isbn}>: {self.title} ({self.duration_min} min)"
    @property
    def is_digital(self) -> bool:
        return True
    @property
    def media_type(self) -> str:
        return "audiobook"
    @classmethod
    def kind(cls) -> str:
        return "audio"
    @property
    def borrowable(self) -> bool:
        return True
    def daily_late_fee(self) -> float:
        return 0.15
    def validate_on_add(self) -> None:
        if self.copies < 0:
            raise ValueError("audio book copies must be non-negative")


# --- Optional Protocol example (kept simple & non-generic) ---
class Downloadable(Protocol):
    def download_link(self) -> str: ...


# --- Core containers ---
@dataclass
class Member:
    member_id: str
    email: str
    role: 'MemberRole' = None
    def max_concurrent_loans(self) -> int:
        return self.role.max_concurrent_loans()

@dataclass
class Loan:
    isbn: str
    member_id: str
    due_date: datetime
    returned: bool = False
    def mark_returned(self) -> None:
        self.returned = True

class Catalog:
    def __init__(self):
        self._items: Dict[str, LibraryItem] = {}
    def add_item(self, item: LibraryItem) -> None:
        if item.isbn in self._items:
            raise DuplicateBookError(f"ISBN already exists: {item.isbn}")
        item.validate_on_add()
        self._items[item.isbn] = item
    def get_item(self, isbn: str) -> Optional[LibraryItem]:
        return self._items.get(isbn)

class LoanDesk:
    def __init__(self, catalog: Catalog):
        self.catalog = catalog
        self.loans: List[Loan] = []
    def active_loans_for(self, member: Member) -> List[Loan]:
        return [L for L in self.loans if (L.member_id == member.member_id and not L.returned)]
    def checkout(self, member: Member, item: LibraryItem) -> Loan:
        if len(self.active_loans_for(member)) >= member.max_concurrent_loans():
            raise LibraryError("concurrent loan limit reached")
        if not item.can_checkout():
            raise NonBorrowableError("cannot checkout under current stock policy")
        if not item.borrowable:
            raise NonBorrowableError("item is not borrowable")
        item.copies -= 1
        loan = Loan(isbn=item.isbn, member_id=member.member_id, due_date=item.due_date_from_today())
        self.loans.append(loan)
        return loan
    def checkin(self, loan: Loan) -> None:
        if not loan.returned:
            it = self.catalog.get_item(loan.isbn)
            if it:
                it.copies += 1
            loan.mark_returned()

## 1) Make `LibraryItem` truly abstract

Prove that `LibraryItem` cannot be instantiated directly. Write a quick try/except demonstrating that creating `LibraryItem('x','y')` raises a `TypeError` because of abstract methods.

In [ ]:
# Your code here
# Demonstrate TypeError on instantiation of abstract class
try:
    test_item = LibraryItem('x', 'y')
except TypeError as e:
    print(f'LibraryItem cannot be instantiated directly due to abstract methods, see error below... \n{e}')

## 2) Abstract property practice

Add an **abstract property** `media_type` to `LibraryItem` and implement it in all concrete subclasses with strings like `'print'`, `'ebook'`, `'audiobook'`.

In [ ]:
# Your code here
# Add @property @abstractmethod def media_type(self) -> str: ...

# Added the following code in the scaffolding of the notebook, in the LibraryItem, PrintedBook, EBook, and AudioBook classes respectively. 

    # @property
    # @abstractmethod
    # def media_type(self) -> str:
        # """Media type as string."""

    # def media_type(self) -> str:
        # return "print"

    # def media_type(self) -> str:
        # return "ebook"

    # def media_type(self) -> str:
        # return "audiobook"

## 3) Abstract classmethod

Add an abstract `@classmethod def kind(cls) -> str` to `LibraryItem` returning a short identifier (e.g., `'book'`). Implement it for the concrete subclasses.

In [ ]:
# Your code here
# classmethod kind() -> str on LibraryItem and overrides

# Added the following code in the scaffolding of the notebook, in the LibraryItem, PrintedBook, EBook, and AudioBook classes respectively. 

    # @classmethod
    # @abstractmethod
    # def kind(cls) -> str:
        # """Briefly identify the kind of book."""

    # @classmethod
    # def kind(cls) -> str:
        # return "printed"

    # @classmethod
    # def kind(cls) -> str:
        # return "ebook"

    # @classmethod
    # def kind(cls) -> str:
        # return "audio"

## 4) Template method using abstract hook

Create a template method `receipt_line(self) -> str` on `LibraryItem` that uses `self.describe()` and `self.loan_period_days()`. Show that each subclass inherits the same template but outputs different text due to overrides.

In [ ]:
# Your code here
# Implement receipt_line in LibraryItem using abstract hooks

# Added the following code in the scaffolding of the notebook, in the LibraryItem class.

    # def receipt_line(self) -> str:
        # return f"{self.describe()} — Due in {self.loan_period_days()} days"

# Shown -> each subclass inherits identical templates, outputs different text when subclass objects call .receipt_line() due to overriding.
pb = PrintedBook("137F", "Darkly Dreaming Dexter", 2)
eb = EBook("274N", "Dune : Messiah", 1, 5.5)
ab = AudioBook("396R", "Fire And Blood", 2, 120)
for item in [pb, eb, ab]:
    print(item.receipt_line())

## 5) Virtual subclass registration

Create a new class `PDFPamphlet` **without** inheriting from `LibraryItem`, but `register` it as a virtual subclass using `LibraryItem.register(PDFPamphlet)`. Implement the required interface manually. Show that `isinstance(pdf, LibraryItem)` returns `True` after registration.

In [ ]:
# Your code here
# Define PDFPamphlet, register as virtual subclass, demonstrate isinstance

class PDFPamphlet:
    def __init__(self, isbn: str, title: str, copies: int = 1, size_mb: float = 0.5):
        self.isbn = isbn
        self.title = title
        self.copies = copies
        self.size_mb = size_mb
    def loan_period_days(self) -> int:
        return 7
    def describe(self) -> str:
        return f"PDFPamphlet<{self.isbn}>: {self.title} ({self.size_mb:.1f} MB)"
    @property
    def is_digital(self) -> bool:
        return True
    @property
    def media_type(self) -> str:
        return "pamphlet"
    @classmethod
    def kind(cls) -> str:
        return "pdfpamphlet"
    @property
    def borrowable(self) -> bool:
        return self.copies > 0
    def daily_late_fee(self) -> float:
        return 0.05
    def validate_on_add(self) -> None:
        if self.copies < 0:
            raise ValueError("pdf pamphlet copies must be non-negative")
    def can_checkout(self) -> bool:
        return self.copies > 0
    def due_date_from_today(self) -> datetime:
        return datetime.now() + timedelta(days=self.loan_period_days())
    def full_label(self) -> str:
        return f"[{self.media_type}] {self.title} — {self.isbn}"
    def receipt_line(self) -> str:
        return f"{self.describe()} — Due in {self.loan_period_days()} days"

LibraryItem.register(PDFPamphlet)

# Shown -> PDFPamphlet is an instance of LibraryItem since printing the result of isinstance(pdf, LibraryItem) will print True.
pdf = PDFPamphlet("857PVW", "Instructions", 1, 1.2)
print(isinstance(pdf, LibraryItem)) 

## 6) Abstract property for availability

Add an abstract property `borrowable: bool` to `LibraryItem`. For `PrintedBook` and `AudioBook`, return `True`. For `EBook`, return `True` if `copies >= 0`. Demonstrate a check before `LoanDesk.checkout` that raises `NonBorrowableError` if `borrowable` is False.

In [ ]:
# Your code here
# Add property and demonstrate guarding behavior

# Added the following code in the scaffolding of the notebook, in the LibraryItem, LoanDesk, PrintedBook, EBook, and AudioBook classes respectively. 

    # @property
    # @abstractmethod
    # def borrowable(self) -> bool:
        # """Whether or not the item is borrowable."""

    # THIS IS IN THE LOANDESK.CHECKOUT METHOD SPECIFICALLY, AS A PRE-CHECK IN THE LINES PRECEEDING THE MAIN BODY OF THE FUNCTION
    # if not item.borrowable:
        # raise NonBorrowableError("item is not borrowable")

    # def borrowable(self) -> bool:
        # return True

    # def borrowable(self) -> bool:
        # return self.copies >= 0

    # def borrowable(self) -> bool:
        # return True

# Demonstrating that the pre-check added to LoanDesk.checkout successfully raises NonBorrowbleError for objects with False borrowable property.
pb.copies = 0 # reusing the test object, pb, from exercise 4.
try:
    desk = LoanDesk(Catalog())
    desk.catalog.add_item(pb)
    member = Member("m143", "test@example.com", StudentRole()) # adjusted this line to account for changes made to Member class (exercise 11).
    desk.checkout(member, pb)
except NonBorrowableError as e:
    print("NonBorrowableError correctly raised for object with False borrowable property!")

## 7) EBook implements Downloadable Protocol

Implement `download_link(self) -> str` on `EBook` to satisfy `Downloadable`. Write a function `offer_download(x)` that accepts a `Downloadable` and returns its link. Show duck-typed use with an `EBook` instance.

In [ ]:
# Your code here
# EBook.download_link and offer_download(downloadable)

# Added the following code in the scaffolding of the notebook, in the EBook class.

    # def download_link(self) -> str:
        # return f"https://downloads.example.com/{self.isbn}.epub"

def offer_download(x: Downloadable) -> str:
    return x.download_link()

# Shown -> usage of the function with an EBook instance.
eb_download = EBook("987X", "Database Design And Modeling", copies=1, file_size_mb=19)
print(offer_download(eb_download))

## 8) Protocol vs ABC (short reflection)

In a short markdown cell, explain the difference between using an ABC and a Protocol for “interfaces” in Python, and when you might pick one over the other in this project.

In [ ]:
# ABCs are used to enforce implementation inheritance and prevent instantiation unless all abstract methods are defined, while Protocols 
# allow structural ("duck typing"), as in objects only need the right methods and attributes with no explicit inheritance. Protocols are useful 
# for third-party types that can't inherit from a  ABC directly. For this project, ABCs are ideal for validating core object types while Protocols
# are ideal to help integrate third-party types easily if they have the necessary attributes.

## 9) `LoanDesk` typed for abstraction

Refactor type hints in `LoanDesk` to accept `LibraryItem` rather than concrete classes everywhere. Explain (markdown) why depending on the abstract type improves flexibility.

In [ ]:
# Your code here
# Refactored type hints should be reflected in the scaffold.

# Depending on LibraryItem as an abstract type allows the system to work with any concrete or registered virtual subclass, 
# making code more reusable and easier to integrate with future types without changing LoanDesk.

## 10) Abstract fee policy

Add an abstract method `daily_late_fee(self) -> float` to `LibraryItem`. Implement fees:
- PrintedBook: 0.25
- EBook: 0.10
- AudioBook: 0.15
Add a concrete `late_fee(self, days_late: int) -> float` in `LibraryItem` that multiplies days by `daily_late_fee()`.

In [ ]:
# Your code here
# Add abstract daily_late_fee and concrete late_fee to LibraryItem and overrides

# Added the following code in the scaffolding of the notebook, in the LibraryItem, PrintedBook, EBook, and AudioBook classes respectively. 

    # @abstractmethod
    # def daily_late_fee(self) -> float:
        # """Late fee per day for the given book."""
        
    # def late_fee(self, days_late: int) -> float:
        # return self.daily_late_fee() * days_late

    # def daily_late_fee(self) -> float:
        # return 0.25

    # def daily_late_fee(self) -> float:
        # return 0.10

    # def daily_late_fee(self) -> float:
        # return 0.15

## 11) ABC for Member roles

Create an abstract base `MemberRole(ABC)` with `max_concurrent_loans(self) -> int`. Implement `StudentRole` (5) and `StaffRole` (10). Modify `Member` to hold a `role: MemberRole` and delegate `max_concurrent_loans()` to it. Keep the implementation single-inheritance (no mixins).

In [ ]:
# Your code here
# Define roles and update Member

class MemberRole(ABC):
    @abstractmethod
    def max_concurrent_loans(self) -> int:
        pass

class StudentRole(MemberRole):
    def max_concurrent_loans(self) -> int:
        return 5

class StaffRole(MemberRole):
    def max_concurrent_loans(self) -> int:
        return 10

## 12) Prevent partial implementations

Create a subclass `BrokenItem(LibraryItem)` that **forgets** to implement one abstract member. Show that instantiating it raises `TypeError`. Then fix it by implementing the missing member.

In [ ]:
# Your code here
# Show failing instantiation then the fix

# Shown -> borrowable is not implemented, this class is broken
class BrokenItem(LibraryItem):
    def loan_period_days(self) -> int:
        return 1
    def describe(self) -> str:
        return "broken"
    @property
    def is_digital(self) -> bool:
        return False
    @property
    def media_type(self) -> str:
        return "broken"
    @classmethod
    def kind(cls) -> str:
        return "broken"
    def daily_late_fee(self) -> float:
        return 1.0
    def validate_on_add(self) -> None:
        pass

# Shown -> attempting to instantiate BrokenItem raises a TypeError
try:
    bitem = BrokenItem("err0r56", "crash", 1)
except TypeError as e:
    print("BrokenItem does not instantiate, missing borrowable abstract property.")

# Shown -> The same class structure with borrowable implemented is "fixed", will instantiate.
class FixedItem(BrokenItem):
    @property
    def borrowable(self) -> bool:
        return True

fitem = FixedItem("ret439", "notCrash", 1)

## 13) Abstract validation hook

Add an abstract hook `validate_on_add(self) -> None` to `LibraryItem` and override it in each subclass to enforce a simple constraint (e.g., `copies >= 0`). Modify `Catalog.add_item` to call `item.validate_on_add()` before insertion.

In [ ]:
# Your code here
# Add validate_on_add to classes and call from Catalog.add_item

# Added the following code in the scaffolding of the notebook, in the LibraryItem, PrintedBook, EBook, and AudioBook classes respectively.

    # @abstractmethod
    # def validate_on_add(self) -> None:
        # """Validate a book when adding."""

    # def validate_on_add(self) -> None:
        # if self.copies < 0:
            # raise ValueError("printed book copies must be non-negative")

    # def validate_on_add(self) -> None:
        # if self.copies < 0:
            # raise ValueError("ebook copies must be non-negative")

    # def validate_on_add(self) -> None:
        # if self.copies < 0:
            # raise ValueError("audio book copies must be non-negative")

# Replaced the following older code snippet inside Catalog.add_item() with the newer code; the call item.validate_on_add() .

        # if item.copies < 0:
            # raise ValueError("copies must be non-negative")

## 14) Minimal adapter via ABC registration

Suppose you receive third-party objects with attributes `code`, `name`, `stock` that you want to treat as `LibraryItem`. Write a light Adapter class that **implements** the `LibraryItem` API and delegates to the 3rd-party object, then register it (or the 3rd-party class) appropriately. Show it working with `LoanDesk.checkout`.

In [ ]:
# Your code here
# Implement Adapter that conforms to LibraryItem contract

class ThirdPartyBook:
    def __init__(self, code, name, stock):
        self.code = code
        self.name = name
        self.stock = stock

class Adapter(LibraryItem):
    def __init__(self, obj: ThirdPartyBook):
        super().__init__(obj.code, obj.name, obj.stock)
        self.obj = obj
    def loan_period_days(self) -> int:
        return 14
    def describe(self) -> str:
        return f"ThirdPartyBook<{self.obj.code}>: {self.obj.name} (stock={self.obj.stock})"
    @property
    def is_digital(self) -> bool:
        return False
    @property
    def media_type(self) -> str:
        return "thirdparty"
    @classmethod
    def kind(cls) -> str:
        return "thirdparty"
    @property
    def borrowable(self) -> bool:
        return self.obj.stock > 0
    def daily_late_fee(self) -> float:
        return 0.20
    def validate_on_add(self) -> None:
        if self.obj.stock < 0:
            raise ValueError("adapted book stock must be non-negative")

# Demo
tp = ThirdPartyBook("ERC82", "Test 3rd Party Book", 3)
adapted_item = Adapter(tp)
cat = Catalog()
cat.add_item(adapted_item)
member = Member("x", "x@example.com", StudentRole())
desk = LoanDesk(cat)
print(desk.checkout(member, adapted_item).isbn)

## 15) Unit test: abstract contract

Using `unittest`, write tests that assert:
- `LibraryItem` instantiation fails
- All concrete classes implement `loan_period_days` and `describe`
- `late_fee` uses the subclass-specific `daily_late_fee` values

In [ ]:
# Your code here
import unittest

class TestWeek9ABCs(unittest.TestCase):
    def test_abstract_instantiation(self):
        with self.assertRaises(TypeError):
            LibraryItem("NCOS33", "Li")

    def test_concretes_implement_contract(self):
        pb = PrintedBook("CNC46", "Pr")
        eb = EBook("CMDL68", "Eb")
        ab = AudioBook("DKV68", "Au")
        for item in [pb, eb, ab]:
            self.assertEqual(type(item.loan_period_days()), int)
            self.assertIsInstance(item.describe(), str)

    def test_late_fee_polymorphism(self):
        pb = PrintedBook("NCOD34", "Pr")
        eb = EBook("OCNS65", "Eb")
        ab = AudioBook("DSOVN55", "Au")
        self.assertEqual(pb.late_fee(2), 0.50)
        self.assertEqual(eb.late_fee(2), 0.20)
        self.assertEqual(ab.late_fee(2), 0.30)

# # To run tests in notebook:
unittest.main(argv=['-v'], exit=False)

## 16) Swap implementation behind the ABC

Write a function `checkout_any(desk: LoanDesk, member: Member, item: LibraryItem)` that works for **any** `LibraryItem` or registered virtual subclass. Demonstrate with a `PDFPamphlet` instance (from Ex. 5) and a normal `PrintedBook`.

In [ ]:
# Your code here
def checkout_any(desk: LoanDesk, member: Member, item: LibraryItem):
    return desk.checkout(member, item)

# Demo with virtual subclass and a real subclass
cat = Catalog()
pb = PrintedBook("101FG", "Another Test Book", 1)
pp = PDFPamphlet("990PE", "More Instructions", 1, 0.9)
cat.add_item(pb)
cat.add_item(pp)
member = Member("MEM8924", "mem@anotherexample.com", StudentRole())
desk = LoanDesk(cat)
print(checkout_any(desk, member, pp))
print(checkout_any(desk, member, pb))

## 17) Abstract property + computed template

Add a template method `full_label()` to `LibraryItem` that returns `f"[{self.media_type}] {self.title} — {self.isbn}"`. Confirm each subclass inherits it and shows the right `media_type` value.

In [ ]:
# Your code here
# Implement full_label in LibraryItem and demonstrate across subclasses

# Added the following code in the scaffolding of the notebook, in the LibraryItem class.

    # def full_label(self) -> str:
        # return f"[{self.media_type}] {self.title} — {self.isbn}"

# Shown -> each sublcass of LibraryItem inherits full_label() correctly, shows proper media type.
for item in [pb, eb, ab, pdf]:
    print(item.full_label())

## 18) issubclass / isinstance with ABCs

Show examples of `issubclass(PrintedBook, LibraryItem)` and `isinstance(EBook(...), LibraryItem)`. After registering a virtual subclass, show `issubclass(PDFPamphlet, LibraryItem)` is `True` as well.

In [ ]:
# Your code here
# Demonstrate issubclass/isinstance facts

print("Is PrintedBook a subclass...", issubclass(PrintedBook, LibraryItem))
print("Is EBook an instance...", isinstance(EBook("ECNCK34", "extra_test_book"), LibraryItem))
print("Is PDFPamphlet a virtual subclass...", issubclass(PDFPamphlet, LibraryItem))

## 19) Inventory report via abstraction

Write `summarize_items(items: list[LibraryItem]) -> list[str]` that uses only abstract methods/properties (`describe`, `media_type`, etc.). Demonstrate polymorphism by passing a mixed list of items.

In [ ]:
# Your code here
def summarize_items(items: list[LibraryItem]) -> list[str]:
    return [f"{item.describe()} — Type: {item.media_type}" for item in items]

# Shown -> summarize_items() is polymorphic since it will apply the same methods onto different objects and return different results.
inventory = [PrintedBook("CBCJ39", "TestPrinted", 1), EBook("CNDD94", "TestElectronic", 2, 1.1), AudioBook("CNKN61", "TestAudio", 1, 40), pdf]
for line in summarize_items(inventory):
    print(line)

## 20) End-to-end scenario under ABC contract

Create a demo that:
- Builds a `Catalog` and `LoanDesk`
- Adds one of each concrete item and one registered virtual subclass instance
- Checks each out to a `Member`
- Prints a small receipt using `receipt_line()` and the computed due dates
Use only `LibraryItem`-level APIs at call sites (no `isinstance` branches).

In [ ]:
# Your code here
# End-to-end demo using the abstract contract only

cat = Catalog()
items = [
    PrintedBook("PDWN45", "Final Print Test", 1),
    EBook("NWOC20", "Final Electronic Test", 1, 2.2),
    AudioBook("AOSO49", "Final Audio Test", 1, 90),
    PDFPamphlet("SPDW35", "Final PDF Test", 1, 0.8)
]
for item in items:
    cat.add_item(item)
desk = LoanDesk(cat)
member = Member("120703", "smclean2@umd.edu", StudentRole())
for item in items:
    loan = desk.checkout(member, item)
    print(f"Checked out: {item.receipt_line()} — Due: {loan.due_date.strftime('%Y-%m-%d')}")